In [ ]:
import re
import pickle
import random

from collections import defaultdict

import torch
import numpy as np

import random

In [ ]:
trans = str.maketrans('(),_-/', '      ','\\"')

#### GET VOCAB AND PAGES

In [ ]:
pages_dict = dict()

word_cnt = 1 # 0 reserved
word2idx = dict()
idx2word = dict()

In [ ]:
with open(r'page.sql', 'r', encoding='utf8') as _if:
    
    for i, line in enumerate(_if):
        
        if (i + 1) % 100 == 0:
            print('processed line {}'.format(i+1))
        
        
        matches = re.findall('\(([^,]*),0,\'(.*?)\',.*?\)', line)
    
        for match in matches:    
            title = match[1].translate(trans).split()
            
            if len(title) > 10:
                continue
            
            for token in title:
                if token not in word2idx:
                    word2idx[token] = word_cnt
                    idx2word[word_cnt] = token
                    
                    word_cnt += 1
            
            title = [word2idx[token] for token in title]
            
            pages_dict[int(match[0])] = title

#### SAVE

In [ ]:
with open('word2idx.pickle', 'wb') as of:
    pickle.dump(word2idx, of)
    
with open('idx2word.pickle', 'wb') as of:
    pickle.dump(idx2word, of)
    
with open('pages_dict.pickle', 'wb') as of:
    pickle.dump(pages_dict, of)    
    

#### LOAD

In [ ]:
with open('word2idx.pickle', 'rb') as _if:
    word2idx = pickle.load(_if)
    
with open('idx2word.pickle', 'rb') as _if:
    idx2word = pickle.load(_if)
    
with open('pages_dict.pickle', 'rb') as _if:
    pages_dict = pickle.load(_if)
    
word_cnt = len(word2idx)

#### GET LINKS

In [ ]:
links_dict = defaultdict(list)

In [ ]:
with open('pagelinks.sql','r', encoding='utf8', errors='ignore') as _if:
    for i, line in enumerate(_if):
    
        if (i + 1) % 1000 == 0:
            print('processed line {}'.format(i+1))
        
        matches = re.findall('\(([0-9]*),0,\'(.*?)\',0\)', line)

        for match in matches:
            if int(match[0]) in pages_dict:
                
                title = match[1].translate(trans).split()
                
                if len(title) > 10:
                    continue
                    
                to_continue = False
                
                for token in title:
                    if token not in word2idx:
                        to_continue = True
                        break
                
                if to_continue:
                    continue      
                        
                title = [word2idx[token] for token in title]
            
                links_dict[tuple(pages_dict[int(match[0])])].append(title)